In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Batch size during training
batch_size = 55

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input/cs480winter2022/5_shot/5_shot/test'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
def train_model(model,
               dataloader,
               criterion,
               optimizer,
               scheduler,
               num_epochs=50,
               plot=True):
    dataset_size = len(dataloader.dataset)
    since = time.time()
    
    #best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    train_acc = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                loss.backward()
                optimizer.step()
            
            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            scheduler.step()
        
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size
            
            train_loss[epoch] = epoch_loss
            train_acc[epoch] = epoch_acc

        if epoch % 10 == 0:
            torch.save(model, f"/kaggle/working/testmodel_epoch{epoch}")

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # plotting
    if plot:
        epoches = list(range(num_epochs))
        
        plt.figure()
        plt.plot(epoches, train_acc, label="train accuracy")
        #plt.plot(epoches, test_acc, label="test accuracy")
        #plt.legend()
        plt.xlabel("epoch")
        plt.ylabel("accuracy")
        plt.title("Accuracy vs Epoch")
        
        plt.figure()
        plt.plot(epoches, train_loss, label="train loss")
        #plt.plot(epoches, test_loss, label="test loss")
        #plt.legend()
        plt.xlabel("epoch")
        plt.ylabel("loss")
        plt.title("Loss vs Epoch")

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model

In [ ]:
resize = transforms.Compose([
    #transforms.Grayscale(num_output_channels=1),
    #transforms.RandomResizedCrop(image_size, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomResizedCrop(image_size, 
                             scale=(0.8, 1.0),
                             ratio=(0.9,1.1),
                             interpolation=transforms.InterpolationMode.BICUBIC),

    #transforms.Resize((image_size, 90), interpolation=transforms.InterpolationMode.BICUBIC),
    #transforms.CenterCrop(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAdjustSharpness(1.25, p=0.8),
    #transforms.RandomVerticalFlip(),
    transforms.ColorJitter( brightness=0.15, contrast=0.15, saturation=0.15, ),
    #transforms.RandomRotation(90),
    transforms.ToTensor(),
    #,
    #transforms.RandomAutocontrast(p=0.5),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
training_data = datasets.ImageFolder("/kaggle/input/cs480winter2022/5_shot/5_shot/train",
                                     transform=resize)
#testing_data = datasets.ImageFolder("/kaggle/input/cs480winter2022/5_shot/5_shot/test")
print(training_data)
#print(testing_data)
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)

# =============> sampling images
for i, data in enumerate(train_loader):
    print(data[0].shape, data[1].shape)
    tensor_image = data[0][0]
    plt.figure()
    plt.imshow(  tensor_image.permute(1, 2, 0)  )


In [ ]:
print(training_data.classes)

In [ ]:
# attempt #1 baseline:
# model = models.resnet50().to(device)
# criterion = nn.CrossEntropyLoss()
# # Observe that all parameters are being optimized
# #optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.7)
# lr = 0.001
# beta1 = 0.5
# optimizer = optim.Adam(model.parameters(), lr=lr, betas=(beta1, 0.999))
# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


# Experiment Tutorial:
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# ConvNet as fixed feature extractor
# model = models.resnet50(pretrained=True)
# for param in model.parameters():
#     param.requires_grad = False

# num_cls = len(training_data.classes) # 22 image classes
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, num_cls)
# model = model.to(device)

# criterion = nn.CrossEntropyLoss()
# lr = 0.0015
# beta1 = 0.5
# optimizer = optim.Adam(model.fc.parameters(), lr=lr,
#                        betas=(beta1, 0.999))

# Finetuning the convnet
model = models.resnet50(pretrained=True)

num_cls = len(training_data.classes) # 22 image classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_cls)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
lr = 0.000125 # slower LR cuz already pre-trained
beta1 = 0.5
optimizer = optim.Adam(model.parameters(), lr=lr,
                       betas=(beta1, 0.999))
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

model = train_model(model,
                    train_loader,
                    criterion,
                    optimizer,
                    exp_lr_scheduler,
                    num_epochs=10)

In [ ]:
# def produce_output(model, dataloader):
#     dataset_size = len(dataloader.dataset)
#     model.eval()

#     for inputs, labels in dataloader:
#         inputs = inputs.to(device)

#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
from PIL import Image

test_size = 517
imgs = torch.stack(
    [TF.to_tensor(
        Image.open(f"/kaggle/input/cs480winter2022/5_shot/5_shot/test/{i}.jpg")
    ) for i in range(test_size) ])

process = transforms.Compose([
    transforms.Resize((image_size+10, 110), interpolation=transforms.InterpolationMode.BICUBIC),
    #transforms.Resize((image_size, 100), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(image_size),
    transforms.RandomAdjustSharpness(1.15, p=1),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
imgs = process(imgs)
print(imgs.shape)

    

In [ ]:

imgs = imgs.to(device)
outputs = model(imgs)
_, preds = torch.max(outputs, 1)
preds = preds.cpu()
df = pd.DataFrame({
    "id": list(range(test_size)),
    "category": [int(training_data.classes[i]) for i in preds]
})
#print(preds)
#print(df.head(10))

print(df.to_csv(index=False))